# Ingesting Housing Market Data from SNB

Data is Housing Price Index from SNB. All values are relative to year 2000 prices (2000 = 100%)

In [29]:
import pandas as pd
import requests
import json

try:
    # Get the data from the SNB API
    url = r"https://data.snb.ch/api/cube/plimoincha/data/json/en"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    
    # Get the JSON data
    data = response.json()
    
    # Initialize an empty list to store all timeseries data
    all_series = []
    
    # Process each timeseries
    for series in data['timeseries']:
        
        # Extract header information
        property_type = series['header'][0]['dimItem']
        data_provider = series['header'][1]['dimItem']
        
        # Extract values | Each row are the values for a different property type and data provider
        values_df = pd.DataFrame(series['values'])
        
        # Add metadata columns
        values_df['property_type'] = property_type
        values_df['data_provider'] = data_provider
        
        # Dont need these columns
        #values_df['unit'] = series['metadata']['unit']
        #values_df['frequency'] = series['metadata']['frequency']
        
        # Append to our list
        all_series.append(values_df)
    
    # Combine all series into one DataFrame
    base_df = pd.concat(all_series, ignore_index=True)
    
    # Convert date column to datetime
    base_df['date'] = pd.to_datetime(base_df['date'], format='%Y')
    
    # Convert value column to numeric
    base_df['value'] = pd.to_numeric(base_df['value'])
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    base_df = None
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    base_df = None
except Exception as e:
    print(f"Error processing data: {e}")
    base_df = None

base_df.head()

,date,value,property_type,data_provider
0,2017-01-01,94.052037,Residential property prices - Privately owned ...,Swiss Federal Statistical Office - Transaction...
1,2018-01-01,96.735820,Residential property prices - Privately owned ...,Swiss Federal Statistical Office - Transaction...
2,2019-01-01,100.000000,Residential property prices - Privately owned ...,Swiss Federal Statistical Office - Transaction...
3,2020-01-01,102.185954,Residential property prices - Privately owned ...,Swiss Federal Statistical Office - Transaction...
4,2021-01-01,107.784384,Residential property prices - Privately owned ...,Swiss Federal Statistical Office - Transaction...


In [5]:
print("unique property types:")
for type in base_df['property_type'].unique():
    print(type)
print('='*100)

print("unique data providers:")
for provider in base_df['data_provider'].unique():
    print(provider)
print('='*100)

unique property types:
Residential property prices - Privately owned apartments
Residential property prices - Single-family houses
Residential property prices - Apartment buildings (residential investment property)
Rents - Rental housing units
Rents - Office space
Rents - Industrial and commercial space
Rents - Retail space
unique data providers:
Swiss Federal Statistical Office - Transaction price
Fahrländer Partner - Transaction price
IAZI - Transaction price
Wüest Partner - Asking price
Wüest Partner - Transaction price


All data is stacked one collumn for the value but there are:
- different property types (Residential property vs. Rents) and (Privately owned appartments vs. Single-family houses)
- different data providers (OFS vs Wuest vs IAZI) and price type (Asking price vs. Transaction price)

for the purpose of predicting the housing market, we want to have a dataframe that has:
- One column for the asking price and one for the transaction price
- One column residential property prices and one for rents by asking price or transaction price
- average together the data providers by price type and property type

Let's start by parsing the data into unique columns



In [30]:
import pandas as pd
import requests
import json

try:
    # Get the data from the SNB API
    url = r"https://data.snb.ch/api/cube/plimoincha/data/json/en"
    response = requests.get(url)
    response.raise_for_status()
    
    data = response.json()
    
    # Initialize an empty list to store all timeseries data
    all_series = []
    
    # Process each timeseries
    for series in data['timeseries']:
        # Extract header information
        property_type = series['header'][0]['dimItem']
        data_provider = series['header'][1]['dimItem']
        
        # Create a DataFrame from the values
        df = pd.DataFrame(series['values'])
        
        # Create a descriptive column name from property type and data provider
        column_name = f"{property_type} - {data_provider}"
        
        # Convert the values to numeric
        df['value'] = pd.to_numeric(df['value'])
        
        # Convert date to datetime
        df['date'] = pd.to_datetime(df['date'], format='%Y')
        
        # Add to list with series information
        all_series.append((df, column_name))
    
    # Start with the first series to get all dates
    base_df = all_series[0][0][['date']].copy()
    
    # Add each series as a new column
    for df, column_name in all_series:
        base_df = base_df.merge(
            df[['date', 'value']], 
            on='date', 
            how='outer'
        )
        base_df = base_df.rename(columns={'value': column_name})
    
    # Sort by date
    base_df = base_df.sort_values('date')
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    base_df = None
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    base_df = None
except Exception as e:
    print(f"Error processing data: {e}")
    base_df = None

base_df.head()

,date,Residential property prices - Privately owned apartments - Swiss Federal Statistical Office - Transaction price,Residential property prices - Privately owned apartments - Fahrländer Partner - Transaction price,Residential property prices - Privately owned apartments - IAZI - Transaction price,Residential property prices - Privately owned apartments - Wüest Partner - Asking price,Residential property prices - Privately owned apartments - Wüest Partner - Transaction price,Residential property prices - Single-family houses - Swiss Federal Statistical Office - Transaction price,Residential property prices - Single-family houses - Fahrländer Partner - Transaction price,Residential property prices - Single-family houses - IAZI - Transaction price,Residential property prices - Single-family houses - Wüest Partner - Asking price,Residential property prices - Single-family houses - Wüest Partner - Transaction price,Residential property prices - Apartment buildings (residential investment property) - Fahrländer Partner - Transaction price,Residential property prices - Apartment buildings (residential investment property) - IAZI - Transaction price,Residential property prices - Apartment buildings (residential investment property) - Wüest Partner - Transaction price,Rents - Rental housing units - Wüest Partner - Asking price,Rents - Office space - Wüest Partner - Asking price,Rents - Industrial and commercial space - Wüest Partner - Asking price,Rents - Retail space - Wüest Partner - Asking price
0,1970-01-01,NaN,NaN,NaN,37.067421,NaN,NaN,NaN,NaN,36.476339,NaN,NaN,NaN,NaN,34.664977,48.588164,46.120740,57.492516
1,1971-01-01,NaN,NaN,NaN,40.041925,NaN,NaN,NaN,NaN,41.771233,NaN,NaN,NaN,NaN,40.164850,52.454522,54.266010,54.101961
2,1972-01-01,NaN,NaN,NaN,45.192274,NaN,NaN,NaN,NaN,50.331605,NaN,NaN,NaN,NaN,45.606831,59.946112,56.381710,72.492423
3,1973-01-01,NaN,NaN,NaN,51.021098,NaN,NaN,NaN,NaN,59.446265,NaN,NaN,NaN,NaN,50.334182,64.896925,66.659691,76.553432
4,1974-01-01,NaN,NaN,NaN,50.419630,NaN,NaN,NaN,NaN,59.588616,NaN,NaN,NaN,NaN,53.390472,65.482283,63.007929,72.950228


## Defining some cleaning functions

In [35]:
def import_snb_housing_data(url="https://data.snb.ch/api/cube/plimoincha/data/json/en"):
    try:
        # Get the data from the SNB API
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        
        # Initialize an empty list to store all timeseries data
        all_series = []
        
        # Process each timeseries
        for series in data['timeseries']:
            # Extract header information
            property_type = series['header'][0]['dimItem']
            data_provider = series['header'][1]['dimItem']
            
            # Create a DataFrame from the values
            df = pd.DataFrame(series['values'])
            
            # Create a descriptive column name from property type and data provider
            column_name = f"{property_type} - {data_provider}"
            
            # Convert the values to numeric
            df['value'] = pd.to_numeric(df['value'])
            
            # Convert date to datetime
            df['date'] = pd.to_datetime(df['date'], format='%Y')
            
            # Add to list with series information
            all_series.append((df, column_name))
        
        # Start with the first series to get all dates
        base_df = all_series[0][0][['date']].copy()
        
        # Add each series as a new column
        for df, column_name in all_series:
            base_df = base_df.merge(
                df[['date', 'value']], 
                on='date', 
                how='outer'
            )
            base_df = base_df.rename(columns={'value': column_name})
        
        # Sort by date
        base_df = base_df.sort_values('date')
        return base_df
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"Error processing data: {e}")
        return None
    

def remove_sfso_data(df):
    """
    Remove Swiss Federal Statistical Office data from the DataFrame
    """
    # Get columns that don't contain 'Swiss Federal Statistical Office'
    non_sfso_cols = [col for col in df.columns 
                     if 'Swiss Federal Statistical Office' not in col or col == 'date']
    
    return df[non_sfso_cols]

def split_residental_rents(df):

    # Get all columns that contain 'Residential property prices'
    residential_prices = [col for col in df.columns if 'Residential property prices' in col]
    
    # Create a new DataFrame with only the residential prices
    residential_df = pd.concat([df['date'], df[residential_prices]], axis=1)

    # Get all columns that contain 'Rents'
    rents = [col for col in df.columns if 'Rents' in col]

    # Create a new DataFrame with only the rents
    rents_df = pd.concat([df['date'], df[rents]], axis=1)

    return residential_df, rents_df
    

def average_providers(residential_df):
    
    types = {
        'Residential property prices': ['Privately owned apartments', 'Single-family houses', 'Apartment buildings (residential investment property)'],
        'data_provider': ['Fahrländer Partner', 'IAZI', 'Wüest Partner'],
        'price_type': ['Asking price', 'Transaction price'],
    }
    
    # Initialize the averaged DataFrame with the date column
    averaged_df = pd.DataFrame({'date': residential_df['date']})
    
    # For each property type
    for property_type in types['Residential property prices']:
        # For each price type (asking vs transaction)
        for price_type in types['price_type']:
            # Create the pattern to match columns
            pattern = f"{property_type}"  # Simplified pattern matching
            price_pattern = 'Asking price' if 'Asking' in price_type else 'Transaction price'
            
            # Get all columns that match this pattern (will catch different providers)
            matching_cols = [col for col in residential_df.columns 
                           if pattern in col and price_pattern in col]
            
            if matching_cols:
                # Create a temporary DataFrame with just the matching columns
                temp_df = residential_df[matching_cols]
                
                # Count non-NaN values for each row
                #non_nan_count = temp_df.notna().sum(axis=1)
                
                # Calculate row-wise mean (this automatically ignores NaN values)
                row_means = temp_df.mean(axis=1)
                
                # Create column name
                col_name = f"{property_type} - {price_type}"
                
                # Add both the average and the count of providers to the DataFrame
                averaged_df[col_name] = row_means
                #averaged_df[f"{col_name} - Provider Count"] = non_nan_count
    
    return averaged_df

def stack_columns(df):
    """
    Function to stack columns and create a single dataframe with both asking and transaction prices
    aligned by date and property type.
    Returns a dataframe with columns: date, property_type, asking_price, transaction_price
    """
    # Initialize empty lists to store dataframes
    asking_price_dfs = []
    transaction_price_dfs = []
    
    for col in df.columns:
        if col == 'date':
            continue
            
        # Extract the property type by removing the price type suffix
        property_type = col.split(' - ')[0]  # Takes the first part before " - "
        
        # Create temporary dataframe with date and price
        temp_df = pd.concat([df['date'], df[col]], axis=1)
        
        if 'Asking price' in col:
            # Rename the price column to asking_price
            temp_df = temp_df.rename(columns={col: 'asking_price'})
            temp_df['property_type'] = property_type
            asking_price_dfs.append(temp_df)
        elif 'Transaction price' in col:
            # Rename the price column to transaction_price
            temp_df = temp_df.rename(columns={col: 'transaction_price'})
            temp_df['property_type'] = property_type
            transaction_price_dfs.append(temp_df)
    
    # Stack all asking price dataframes
    asking_prices_df = pd.concat(asking_price_dfs, ignore_index=True)
    # Stack all transaction price dataframes
    transaction_prices_df = pd.concat(transaction_price_dfs, ignore_index=True)
    
    # Merge the two dataframes on date and property_type
    combined_df = pd.merge(
        asking_prices_df, 
        transaction_prices_df, 
        on=['date', 'property_type'], 
        how='outer'
    )
    
    # Sort by date and property_type
    combined_df = combined_df.sort_values(['date', 'property_type'])
    
    # Rename Appartement buildings value
    combined_df['property_type'] = combined_df['property_type'].replace({'Apartment buildings (residential investment property)' : 'Appartment buildings'})
    
    return combined_df

def join_rents(combined_df, rents_df):
     
     # First let's rename the rents DF columns
     for col in rents_df.columns:
          if col == 'date':
               continue
          name = col.split('-')
          name = ('-').join(name[0:2])
          rents_df = rents_df.rename(columns={col: name})
     
     # Now let's join both DFs together

     housing_df = pd.merge(
          left = combined_df, 
          right = rents_df, 
          on = 'date',
          how = 'outer'
          )
     
     return housing_df
    
def get_clean_housing_data(url="https://data.snb.ch/api/cube/plimoincha/data/json/en"):
    base_df = import_snb_housing_data(url)
    cleaned_df = remove_sfso_data(base_df)
    residential_df, rents_df = split_residental_rents(cleaned_df)
    averaged_df = average_providers(residential_df)
    combined_df = stack_columns(averaged_df)
    housing_df = join_rents(combined_df, rents_df)
    
    return housing_df

In [36]:
housing_df = get_clean_housing_data()
housing_df

,date,asking_price,property_type,transaction_price,Rents - Rental housing units,Rents - Office space,Rents - Industrial and commercial space,Rents - Retail space
0,1970-01-01,NaN,Appartment buildings,NaN,34.664977,48.588164,46.120740,57.492516
1,1970-01-01,37.067421,Privately owned apartments,NaN,34.664977,48.588164,46.120740,57.492516
2,1970-01-01,36.476339,Single-family houses,NaN,34.664977,48.588164,46.120740,57.492516
3,1971-01-01,NaN,Appartment buildings,NaN,40.164850,52.454522,54.266010,54.101961
4,1971-01-01,40.041925,Privately owned apartments,NaN,40.164850,52.454522,54.266010,54.101961
...,...,...,...,...,...,...,...,...
160,2023-01-01,187.658501,Privately owned apartments,240.258694,142.719131,118.597196,117.251560,111.548723
161,2023-01-01,189.898192,Single-family houses,213.819478,142.719131,118.597196,117.251560,111.548723
162,2024-01-01,NaN,Appartment buildings,163.797819,149.401432,117.903731,119.401059,111.641745
163,2024-01-01,188.501891,Privately owned apartments,247.621496,149.401432,117.903731,119.401059,111.641745


## Plotting the Data and Descriptive Statistics